# Trabalho 2 IART: Aprendizagem Supervisionada - Regressão
## Introdução
O futebol é possivelmente o desporto mais popular de sempre com milhões de adeptos e um igual número de jogadores dado ser talvez um dos desportos mais fáceis de jogar que existe.

Para além disso, este desporto movimenta uma grande quantidade de dinheiro quer pela venda de bilhetes e _merchandise_ quer pelas transferências de jogadores. Outro lado desta economia é o mercado das apostas desportivas que é cada vez mais procurado por adeptos. Sabe-se que os apostadores profissionais acertam na sua aposta cerca de 53% das vezes.

Sendo assim, o objetivo deste trabalho é prever o resultado de jogos de maneira a criar um sistema que recomenda o resultado do jogo em que apostar, preferivelmente com uma precisão superior à dos apostadores profissionais.

## Descrição do _dataset_
O _dataset_ inclui informação acerca de 25000 partidas de futebol em ligas de 11 países europeus realizadas nas épocas de 2008 a 2016 e acerca de 10000 jogadores. Este inclui ainda informação sobre os _odds_ fornecidos por vários _sites_ de apostas para os vários jogos.
Mais informação pode ser encontrada [aqui](https://www.kaggle.com/hugomathien/soccer).

## Aquisição e exploração de dados
O _dataset_ que irá ser usado corresponde a uma base de dados sqlite que inclui as tabelas a seguir apresentadas e que contêm a seguinte informação:

In [20]:
import sqlite3

conn = sqlite3.connect('soccer.db')
cursor = conn.cursor()
cursor.execute("select name from sqlite_master where type='table'; ")
cursor.fetchall()

[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]

* **Country:**  Países para os quais existe informação.
* **League:** Ligas de futebol dos vários países incluídos na tabela _Country_.
* **Match:** Informação sobre as várias partidas de futebol que inclui estatísticas do jogo, _odds_, bem como os jogadores que participaram.
* **Player:** Informação sobre os vários jogadores da base de dados, incluindo nome, data de nascimento, altura e peso.
* **Player_Attributes:** Habilidades de cada um dos jogadores tiradas do jogo FIFA.
* **Team:** Informação acerca de cada uma das equipas, incluindo o seu nome.
* **Team_Attributes:** Habilidades de cada uma das equipas tiradas do jogo FIFA.

Dado tratar-se de um _dataset_ extenso (~300MB) apenas serão consideradas partidas jogadas na English Premier League, já que corresponde a uma liga com um número razoável de equipas e jogos. Sendo assim, os passos seguintes têm como objetivo extrair informação acerca desta liga.

In [23]:
import pandas as pd
from lxml import etree as et

matches = pd.read_sql_query("select Match.id, League.name, Match.season, Match.possession, Match.card, Match.foulcommit as foul, T1.team_long_name as home_team, T1.team_api_id as home_team_id, Match.home_team_goal, T2.team_api_id as away_team_id, away_team_goal, stage, date, B365H, B365D, B365A from League, Match on (League.id = Match.league_id), Team as T1 on (T1.team_api_id = Match.home_team_api_id), Team as T2 on (T2.team_api_id = Match.away_team_api_id) where League.name = 'England Premier League'", conn)
matches.head()

# Get possession XML data
matches['possession'] = matches['possession'].apply(lambda x: et.fromstring(x))
matches['card'] = matches['card'].apply(lambda x: et.fromstring(x))
matches['foul'] = matches['foul'].apply(lambda x: et.fromstring(x))

Algumas colunas na tabela __Match__ contém valores em XML,como a coluna _possession_. Portanto para obter os valores absolutos da posse de bola de cada equipa é necessário realizar algumas operações.

In [161]:
possession = et.Element("possession")

for id, row in zip(matches['id'], matches['possession']):
    #print(et.tostring(row, pretty_print=True).decode())
    if len(row.getchildren()):
        match = et.SubElement(possession, 'match')
        match.set('id', str(id))
        for child in row[-1].getchildren():
            el = et.fromstring(et.tostring(child))
            match.append(el)

file = open('possession.xml', 'w+')
file.write(et.tostring(possession, pretty_print=True).decode())

idx = 0
for _, element in et.iterparse("possession.xml", tag='match'):
#     print('Match_id: ' + element.get('id'))
#     print('HOME_POS: %s -- AWAY_POS: %s\n' % (element.findtext('homepos'), element.findtext('awaypos')))
    matches.at[idx, "HP"] = element.findtext('homepos')
    matches.at[idx, "AP"] = element.findtext('awaypos')
    
    idx += 1
    element.clear(keep_tail=True)
    
matches.head()

,id,name,season,possession,card,home_team,home_team_goal,away_team,away_team_goal,stage,date,B365H,B365D,B365A,HP,AP
0,1729,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x11585c4b0>], [], [...",10260,1,10261,1,1,2008-08-17 00:00:00,1.29,5.5,11.00,55,45
1,1730,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...",[],9825,1,8659,0,1,2008-08-16 00:00:00,1.20,6.5,15.00,66,34
2,1731,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x11585ca50>], [], [...",8472,0,8650,1,1,2008-08-16 00:00:00,5.50,3.6,1.67,46,54
3,1732,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x11585c4b0>], [], [...",8654,2,8528,1,1,2008-08-16 00:00:00,1.91,3.4,4.20,52,48
4,1733,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x11585c4b0>], [], [...",10252,4,8456,2,1,2008-08-17 00:00:00,1.91,3.4,4.33,52,48


In [134]:
import numpy 

card = et.Element("card")

for id, row in zip(matches['id'], matches['card']):
    #print(et.tostring(row, pretty_print=True).decode())
    if len(row.getchildren()):
        match = et.SubElement(card,'match')
        match.set('id', str(id))
        for child in row.getchildren():
            el = et.fromstring(et.tostring(child))
            match.append(el)

file = open('card.xml', 'w+')
file.write(et.tostring(card, pretty_print=True).decode())


matches['HY'] = 0
matches['AY'] = 0
matches['HR'] = 0
matches['AR'] = 0
for _, element in et.iterparse("card.xml", tag='match'):
    id = element.get('id')
    idx = matches[matches['id'] == numpy.int64(id)].index.values.astype(int)[0]
    
    for child in element:
        if(child.findtext('team') == str(matches.at[idx, 'home_team_id'])):
            if(child.findtext('card_type') == 'y'):
                matches.at[idx, 'HY'] += 1
            else:
                matches.at[idx, 'HR'] += 1

        elif(child.findtext('team') == str(matches.at[idx, 'away_team_id'])):
            if(child.findtext('card_type') == 'y'):
                matches.at[idx, 'AY'] += 1
            else:
                matches.at[idx, 'AR'] += 1

    element.clear(keep_tail=True)
matches.head()

,id,name,season,possession,card,home_team,home_team_id,home_team_goal,away_team,away_team_id,...,date,B365H,B365D,B365A,HP,AP,HY,AY,HR,AR
0,1729,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x249e3b86740>], [],...",Manchester United,10260,1,Newcastle United,10261,...,2008-08-17 00:00:00,1.29,5.5,11.00,55,45,3,0,0,0
1,1730,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...",[],Arsenal,9825,1,West Bromwich Albion,8659,...,2008-08-16 00:00:00,1.20,6.5,15.00,66,34,0,0,0,0
2,1731,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x249e3b86740>], [],...",Sunderland,8472,0,Liverpool,8650,...,2008-08-16 00:00:00,5.50,3.6,1.67,46,54,0,2,0,0
3,1732,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x249e3b86180>], [],...",West Ham United,8654,2,Wigan Athletic,8528,...,2008-08-16 00:00:00,1.91,3.4,4.20,52,48,2,1,0,0
4,1733,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x249e3b86180>], [],...",Aston Villa,10252,4,Manchester City,8456,...,2008-08-17 00:00:00,1.91,3.4,4.33,52,48,0,1,0,0


In [25]:
import numpy 

foul = et.Element("foul")

for id, row in zip(matches['id'], matches['foul']):
    #print(et.tostring(row, pretty_print=True).decode())
    if len(row.getchildren()):
        match = et.SubElement(foul,'match')
        match.set('id', str(id))
        for child in row.getchildren():
            el = et.fromstring(et.tostring(child))
            match.append(el)

file = open('foul.xml', 'w+')
file.write(et.tostring(foul, pretty_print=True).decode())


matches['HF'] = 0
matches['AF'] = 0
for _, element in et.iterparse("foul.xml", tag='match'):
    id = element.get('id')
    idx = matches[matches['id'] == numpy.int64(id)].index.values.astype(int)[0]
    
    for child in element:
        if(child.findtext('team') == str(matches.at[idx, 'home_team_id'])):
            matches.at[idx, 'HF'] += 1

        elif(child.findtext('team') == str(matches.at[idx, 'away_team_id'])):
                matches.at[idx, 'AF'] += 1


    element.clear(keep_tail=True)
matches.head()

,id,name,season,possession,card,foul,home_team,home_team_id,home_team_goal,away_team_id,...,date,B365H,B365D,B365A,HY,AY,HR,AR,HF,AF
0,1729,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x119141640>], [], [...","[[[<Element foulscommitted at 0x119141640>], [...",Manchester United,10260,1,10261,...,2008-08-17 00:00:00,1.29,5.5,11.00,0,0,16,11,16,11
1,1730,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...",[],"[[[<Element foulscommitted at 0x119141640>], [...",Arsenal,9825,1,8659,...,2008-08-16 00:00:00,1.20,6.5,15.00,0,0,11,9,11,9
2,1731,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x119141730>], [], [...","[[[<Element foulscommitted at 0x119141640>], [...",Sunderland,8472,0,8650,...,2008-08-16 00:00:00,5.50,3.6,1.67,0,0,13,12,13,12
3,1732,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x119141640>], [], [...","[[[<Element foulscommitted at 0x119141640>], [...",West Ham United,8654,2,8528,...,2008-08-16 00:00:00,1.91,3.4,4.20,0,0,14,13,14,13
4,1733,England Premier League,2008/2009,"[[[], [], [], [], [], [], [], [], [], []], [[]...","[[[], [<Element ycards at 0x119141640>], [], [...","[[[<Element foulscommitted at 0x119141640>], [...",Aston Villa,10252,4,8456,...,2008-08-17 00:00:00,1.91,3.4,4.33,0,0,11,13,11,13


Vamos agora criar três novas colunas que representam para cada jogo o lado vencedor (equipa da casa ou equipa de fora) ou empate.

In [32]:
matches = matches.assign(home_win=lambda matches: matches.apply(lambda row: 1 if row.home_team_goal > row.away_team_goal else 0, axis='columns'),
              draw=lambda matches: matches.apply(lambda row: 1 if row.home_team_goal == row.away_team_goal else 0, axis='columns'),
              away_win=lambda matches: matches.apply(lambda row: 1 if row.home_team_goal < row.away_team_goal else 0, axis='columns'))

Podemos agora calcular a taxa de vitórias, empates e derrotas em relação à equipa que joga em casa.

In [33]:
win_rates = matches.groupby('season') \
                   .mean() \
                   .loc[:, ['home_win', 'draw', 'away_win']]
win_rates

,home_win,draw,away_win
season,,,
2008/2009,0.455263,0.255263,0.289474
2009/2010,0.507895,0.252632,0.239474
2010/2011,0.471053,0.292105,0.236842
2011/2012,0.450000,0.244737,0.305263
2012/2013,0.436842,0.284211,0.278947
2013/2014,0.471053,0.205263,0.323684
2014/2015,0.452632,0.244737,0.302632
2015/2016,0.413158,0.281579,0.305263


Com estes dados podemos concluir que as equipas que jogam em casa têm uma maior possibilidade de vencer a partida.
Cerca de 46% dos jogos acabam com a vitória da equipa da casa, 28% acabam com vitória da equipa de fora e o acontecimento menos comum é o empate que ocorre em 26% dos jogos. Apresenta-se agora um gráfico com a evolução das taxas em estudo:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(111)


home_line = ax.plot(win_rates.home_win, label='Vitórias em casa')
away_line = ax.plot(win_rates.away_win, label='Vitórias equipa de fora')
draw_line = ax.plot(win_rates.draw, label='Empates')
ax.set_xlabel("Época")
ax.set_ylabel("Taxa de vitórias")
plt.title("Taxa de Vitórias", fontsize=16)

# Add the legend locations
home_legend = plt.legend(handles=home_line, loc='upper right', bbox_to_anchor=(1, 1))
ax = plt.gca().add_artist(home_legend)
away_legend = plt.legend(handles=away_line, loc='center right', bbox_to_anchor=(0.95, 0.45))
ax = plt.gca().add_artist(away_legend)
draw_legend = plt.legend(handles=draw_line, loc='center right', bbox_to_anchor=(0.99, 0.06))

Podemos observar no gráfico que cada vez é mais difícil ter uma vitória em casa, parecendo ser cada vez mais provável haver um empate.

Uma medida interessante é quão vantajoso é para uma equipa, jogar em casa (HGA ou *Home Ground Advantage*) definido como a diferença entre a probabilidade da equipa ganhar em casa e a de ganhar fora.

In [ ]:
home_win_rates = matches.groupby(['home_team']) \
                   .home_win \
                   .mean()

away_win_rates = matches.groupby(['away_team']) \
                   .away_win \
                   .mean()

hga = (home_win_rates - away_win_rates).reset_index().rename(columns={0: 'HGA'}).sort_values(by='HGA', ascending=False)
hga.head()

Mais interessante ainda é a variação desta medida ao longo do tempo para os vários clubes.

In [ ]:
big_clubs = ['Liverpool', 'Manchester City', 'Manchester United', 'Chelsea', 'Arsenal']
home_win_rates_5 = matches[matches.home_team.isin(big_clubs)].groupby(['home_team', 'season']).home_win.mean()
away_win_rates_5 = matches[matches.away_team.isin(big_clubs)].groupby(['away_team', 'season']).away_win.mean()

hga_top_5 = home_win_rates_5 - away_win_rates_5

hga_top_5.unstack(level=0)

Apresenta-se a seguir um gráfico que representa a tabela acima.

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 5))
plot = sns.lineplot(x='season', y='HGA', hue='home_team', data=hga_top_5.reset_index().rename(columns={0: 'HGA', 'home_team': 'home_team'}))
plt.legend(loc='lower center', ncol=6, bbox_to_anchor=(0.45, -0.2))
plt.title("HGA Among the top 5 clubs", fontsize=14)

plot.axes.axhline(0, ls='--')

plt.show()

Os resultados são erráticos mas há duas equipas que têm esta medida sempre positiva (Arsenal e Manchester City). Interessantemente, algumas equipas, em algumas épocas ganharam mais jogos fora de casa do que em casa.

## Preparação dos Dados

O objetivo desta fase é preparar os dados para o processo de modelação e treino do modelo.

Para isto, é necessário alterar o formato dos dados já que é preferível que os dados relativos a cada uma das equipas estejam cada um na sua linha para serem tratados pelos algoritmos de Machine Learning. Por outro lado, é preferível calcular os dados relativos a cada uma das equipas com as equipas em linhas separadas.

In [ ]:
stats = matches \
        .assign(date=lambda df: pd.to_datetime(df.date)) \
        .pipe(lambda df: df.dropna(thresh=len(df) - 2, axis=1)) \
        .dropna(axis=0) \
        .sort_values('date') \
        .reset_index(drop=True) \
        .assign(gameId=lambda df: list(df.index + 1))

stats_cols = ['gameId', 'home_team', 'away_team', 'home_team_goal', 'away_team_goal']

stats = stats[stats_cols].copy()
stats.head()

In [ ]:
betting = matches \
            .assign(date=lambda df: pd.to_datetime(df.date)) \
            .pipe(lambda df: df.dropna(thresh=len(df) - 2, axis=1)) \
            .dropna(axis=0) \
            .sort_values('date') \
            .reset_index(drop=True) \
            .assign(gameId=lambda df: list(df.index + 1), 
                    year=lambda df: df.date.apply(lambda row: row.year), 
                    home_win=lambda df: df.apply(lambda row: 1 if row.home_team_goal > row.away_team_goal else 0, axis=1),
                    away_win=lambda df: df.apply(lambda row: 1 if row.away_team_goal > row.home_team_goal else 0, axis=1),
                    result=lambda df: df.apply(
                            lambda row: 'home' if row.home_team_goal > row.away_team_goal else ('draw' if row.home_team_goal == row.away_team_goal else 'away'), axis=1))

betting_cols = ['B365A', 'B365D', 'B365H', 'home_win', 'away_win', 'result', 'home_team', 'away_team', 'gameId']

betting = betting[betting_cols].copy()
betting.head()

In [ ]:
# Probably useless
team_info = matches \
            .assign(date=lambda df: pd.to_datetime(df.date)) \
            .pipe(lambda df: df.dropna(thresh=len(df) - 2, axis=1)) \
            .dropna(axis=0) \
            .sort_values('date') \
            .reset_index(drop=True) \
            .assign(gameId=lambda df: list(df.index + 1), 
                    year=lambda df: df.date.apply(lambda row: row.year), 
                    home_win=lambda df: df.apply(lambda row: 1 if row.home_team_goal > row.away_team_goal else 0, axis=1),
                    away_win=lambda df: df.apply(lambda row: 1 if row.away_team_goal > row.home_team_goal else 0, axis=1),
                    result=lambda df: df.apply(
                            lambda row: 'home' if row.home_team_goal > row.away_team_goal else ('draw' if row.home_team_goal == row.away_team_goal else 'away'), axis=1))

team_info = team_info[['gameId', 'date', 'season', 'home_team', 'away_team']].copy()
team_info.head()

In [ ]:
historic_games = matches \
                .assign(date=lambda df: pd.to_datetime(df.date),
                        gameId=-1,
                        homeWin=lambda df: df.apply(lambda row: 1 if row.home_team_goal > row.away_team_goal else 0, axis='columns'))
historic_games.head()

### Criação de features

TODO: escrever qualquer coisa

In [ ]:
# Define a function which restructures our DataFrame
def create_multiline_df_stats(old_stats_df):
    # Create a list of columns we want and their mappings to more interpretable names
    home_stats_cols = ['home_team', 'home_team_goal', 'away_team_goal']
    
    away_stats_cols = ['away_team', 'home_team_goal', 'away_team_goal']
    
    stats_cols_mapping = ['team', 'goals_for', 'goals_against']
    
    # Create a dictionary of the old column names to new column names
    home_mapping = {old_col: new_col for old_col, new_col in zip(home_stats_cols, stats_cols_mapping)}
    away_mapping = {old_col: new_col for old_col, new_col in zip(away_stats_cols, stats_cols_mapping)}
    
    # Put each team onto an individual row
    multi_line_stats = old_stats_df[['gameId'] + home_stats_cols] \
                        .rename(columns=home_mapping) \
                        .assign(home_game=1) \
                        .append((old_stats_df[['gameId'] + away_stats_cols]) \
                                .rename(columns=away_mapping) \
                                .assign(home_game=0), sort=True) \
                        .sort_values(by='gameId') \
                        .reset_index(drop=True)
    return multi_line_stats

In [ ]:
# Define a function which creates an EMA DataFrame from the stats DataFrame
def create_stats_features_ema(stats, span):
    # Create a restructured DataFrames so that we can calculate EMA
    multi_line_stats = create_multiline_df_stats(stats)
    
    # Create a copy of the DataFrame
    ema_features = multi_line_stats[['gameId', 'team', 'home_game']].copy()
    
    # Get the columns that we want to create EMA for
    feature_names = multi_line_stats.drop(columns=['gameId', 'team', 'home_game']).columns
    
    # Loop over the features
    for feature_name in feature_names:
        feature_ema = (multi_line_stats.groupby('team')[feature_name] # Calculate the EMA
                                                  .transform(lambda row: row.ewm(span=span, min_periods=2)
                                                             .mean()
                                                             .shift(1))) # Shift the data down 1 so we don't leak data
        ema_features[feature_name] = feature_ema # Add the new feature to the DataFrame
    return ema_features

# Apply the function
stats_features = create_stats_features_ema(stats, span=5)
stats_features.tail()

In [ ]:
pd.DataFrame(stats_features.groupby('team')
                           .goals_for
                           .mean()
                           .sort_values(ascending=False)[:10])

### Otimizar o Alpha

In [ ]:
def restructure_stats_features(stats_features):
    non_features = ['home_game', 'team', 'gameId']

    stats_features_restructured = (stats_features.query('home_game == 1')
                                    .rename(columns={col: 'f_' + col + 'Home' for col in stats_features.columns if col not in non_features})
                                    .rename(columns={'team': 'home_team'})
                                    .pipe(pd.merge, (stats_features.query('home_game == 0')
                                                        .rename(columns={'team': 'away_team'})
                                                        .rename(columns={col: 'f_' + col + 'Away' for col in stats_features.columns 
                                                                         if col not in non_features})), on=['gameId'])
                                    .pipe(pd.merge, betting[['gameId', 'result']], on='gameId')
                                    .dropna())
    return stats_features_restructured

restructure_stats_features(stats_features).head()

In [ ]:
def optimise_alpha(features):
    le = LabelEncoder()
    y = le.fit_transform(features.result) # Encode the result from away, draw, home win to 0, 1, 2
    X = features[[col for col in features.columns if col.startswith('f_')]] # Only get the features - these all start with f_
    lr = LogisticRegression()
    
    kfold = StratifiedKFold(n_splits=5)
    ave_cv_score = cross_val_score(lr, X, y, scoring='neg_log_loss', cv=kfold).mean()
    return ave_cv_score

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score

best_score = np.float('inf')
best_span = 0
cv_scores = []

# Iterate over a range of spans
for span in range(1, 100, 3):
    stats_features = create_stats_features_ema(stats, span=span)
    restructured_stats_features = restructure_stats_features(stats_features)
    cv_score = optimise_alpha(restructured_stats_features)
    cv_scores.append(cv_score)
    
    if cv_score * -1 < best_score:
        best_score = cv_score * -1
        best_span = span

In [ ]:
# There must be a problem here, lowest log loss is at a span of 1
plt.plot(list(range(1, 100, 3)), (pd.Series(cv_scores)*-1)) # Plot our results

plt.title("Optimising alpha")
plt.xlabel("Span")
plt.ylabel("Log Loss")
plt.show()

print("Our lowest log loss ({:2f}) occurred at a span of {}".format(best_score, best_span))

In [ ]:
features_stats = create_stats_features_ema(stats, span=best_span)
features_stats.tail(3)

In [ ]:
def create_multiline_df_betting(betting):
    multi_line_odds = betting[['gameId', 'home_team']] \
                      .rename(columns={'home_team': 'team'}) \
                      .append(betting[['gameId', 'away_team']]) \
                      .rename(columns={'away_team': 'team'}) \
                      .sort_values(by='gameId')\
                      .reset_index(drop=True)
    return multi_line_odds

def create_betting_features_ema(betting, span):
    multi_line_odds = create_multiline_df_betting(betting)
    
    ema_features = multi_line_odds[['gameId', 'team']].copy()

    feature_names = multi_line_odds.drop(columns=['gameId', 'team']).columns

    for feature_name in feature_names:
        feature_ema = (multi_line_odds.groupby('team')[feature_name]
                                      .transform(lambda row: row.ewm(span=span, min_periods=2)
                                      .mean()
                                      .shift(1)))

        ema_features[feature_name] = feature_ema
        
    return ema_features

# Create feature_odds df
features_odds = create_betting_features_ema(betting, span=10)
features_odds.tail(3)

# Escolher os algoritmos

Para analisarmos os dados escolhemos os algoritmos Logistic Regression, K-Nearest-Neighbours e Support Vector Machines pois são os mais adequados para os dados que temos. 

In [ ]:
def create_feature_df(df=None):
    ##########################
    ## Create segmented dfs ##
    ##########################
    
    if df is None:
        df = create_df('data/epl_data.csv')

    stats_cols = ['gameId', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 
                                'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

    betting_cols = ['B365A', 'B365D', 'B365H', 'BWA', 'BWD', 'BWH', 'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 
                            'BbAv>2.5', 'BbAvA', 'BbAvAHA', 'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 
                            'BbMxA', 'BbMxAHA', 'BbMxAHH', 'BbMxD', 'BbMxH', 'BbOU', 'Day', 'Div', 'IWA', 'IWD', 
                            'IWH', 'LBA', 'LBD', 'LBH', 'Month', 'VCA', 'VCD', 'VCH', 'Year', 
                            'homeWin', 'awayWin', 'result', 'HomeTeam', 'AwayTeam', 'gameId']
    betting_cols = [col for col in betting_cols if col in betting_cols and col in df]


    team_info = df[['gameId', 'Date', 'season', 'HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee']].copy()                  
    stats = df[stats_cols].copy()
    betting = df[betting_cols].copy()
    historic_games = create_historic_games_df('data/historic_games_pre2005.csv')
        
    all_games = (historic_games.append(df[historic_games.columns])
                             .reset_index(drop=True)
                             .assign(awayWin=lambda df: df.apply(lambda row: 1 if row.FTAG > row.FTHG else 0, axis='columns'))
                             .pipe(lambda df: win_pc(df, 5, "HomeTeam", "homeWin", "homeWinPc5"))
                             .pipe(lambda df: win_pc(df, 38, "HomeTeam", "homeWin", "homeWinPc38"))
                             .pipe(lambda df: win_pc(df, 5, "AwayTeam", "awayWin", "awayWinPc5"))
                             .pipe(lambda df: win_pc(df, 38, "AwayTeam", "awayWin", "awayWinPc38"))
                             .assign(gameIdHistoric=lambda df: list(range(1, len(df) + 1))))

    ########################
    ## Create feature dfs ##
    ########################

    print("Creating all games feature DataFrame")
    features_all_games = create_all_games_features(all_games)

    print("Creating stats feature DataFrame")
    features_stats = create_stats_features_ema(stats, span=49)

    print("Creating odds feature DataFrame")
    features_odds = create_betting_features_ema(betting, span=10)

    print("Creating market values feature DataFrame")
    features_market_values = create_market_values_features(df) # This creates a df with one game per row

    all_games_cols = ['Date', 'gameId', 'team', 'season', 'homeGame', 'homeWinPc38', 'homeWinPc5', 'awayWinPc38', 'awayWinPc5', 'eloFor', 'eloAgainst', 'wtEloGoalsFor', 'wtEloGoalsAgainst']

    features_multi_line = (features_all_games[all_games_cols]
                                             .pipe(pd.merge, features_stats, on=['gameId', 'team'])
                                             .pipe(pd.merge, features_odds, on=['gameId', 'team']))
    print("Filling NAs")

    # Put each instance on an individual row
    features_with_na = put_features_on_one_line(features_multi_line)

    market_val_feature_names = ['attMktH%', 'attMktA%', 'midMktH%', 'midMktA%', 'defMktH%', 'defMktA%', 'gkMktH%', 'gkMktA%', 'totalMktH%', 'totalMktA%']
    print("Merging stats, odds and market values into one features DataFrame")

    # Merge our team values dataframe to features and result from df
    features_with_na = (features_with_na.pipe(pd.merge, features_market_values[market_val_feature_names + ['gameId']].rename(columns={col: 'f_' + col for col in market_val_feature_names}), on='gameId')
                                        .pipe(pd.merge, df[['HomeTeam', 'AwayTeam', 'gameId', 'result', 'B365A', 'B365D', 'B365H']], on=['HomeTeam', 'AwayTeam', 'gameId'])
                                        .drop(columns='homeGame'))

    # Drop NAs from calculating the rolling averages - don't drop Win Pc 38 and Win Pc 5 columns
    features = features_with_na.dropna(subset=features_with_na.drop(columns=[col for col in features_with_na.columns if 'WinPc' in col]).columns)

    # Fill NAs for the Win Pc columns
    features = features.fillna(features.mean())

    features = features.rename(columns={
        'B365H': 'f_homeOdds',
        'B365A': 'f_awayOdds',
        'B365D': 'f_drawOdds'
        })

    print("Complete.")
    return features

features = create_feature_df()

